Micah Gale

19 Sep 25

# NE 536: HomeWork 1


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Problem 1.1 Plasma Parameters

Given:

* $T_{e^-} = 1 keV$
* $n = 5 \times 10^{18} \frac{1}{m^3}$
* $|\vec{B}| = 1 T$

In [ ]:
# store inputs as constants
TEMP_E = 1_000 # [eV]
DENSITY = 5e18 # [m^-3]
FIELD_ST = 1.0 # T

Calculate:
* Debye Length
* Plasma frequency
* electron collision frequency

### Debye Length

$\lambda_d = \sqrt{
\frac{\epsilon_0 T_e}{ne^2}}$

We have density, $n$, and electron temperature, $T_{e^-}$, so we just need the constants: vacuum permittivity, $\epsilon_0$, and the elementary charge, $e$.

I used:

* $\epsilon_0 = 8.854 \times 10^{-12}\frac{F}{m}$ [1]
* $e = 1.602\times 10^{-19} C$ [2]

[1] “Vacuum permittivity,” Wikipedia. July 20, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Vacuum_permittivity&oldid=1301610312

[2] “Elementary charge,” Wikipedia. June 03, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Elementary_charge&oldid=1293751741



In [ ]:
EPSILON_0 = 8.854e-12 #[F/m]
ELEM_CHARGE = 1.602e-19 #[C]

In [ ]:
temp_e_j = TEMP_E * ELEM_CHARGE # convert to Joules
debye_l = np.sqrt((EPSILON_0 * temp_e_j) / (DENSITY * ELEM_CHARGE ** 2))
print(f"Debye Length for plasma at electron temperature {TEMP_E / 1000:.1f} keV, and a density of " 
      f"{DENSITY:.1e} m^-3 is:\n {debye_l* 1000:.3f} mm")

### Plasma Frequency

The plasma frequency, $\omega_p$ is given by:

$\omega_p = \sqrt{\frac{ne^2}{m_e\epsilon_0}}$

We just need the mass of an electron, $m_e$ now:

* $m_e = 9.10938 \times 10^{-31} kg$  [3]

[3] “Electron mass,” Wikipedia. May 26, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Electron_mass&oldid=1292442002


In [ ]:
MASS_E = 9.10938e-31 #[kg]

plasma_ang_freq = np.sqrt((density * ELEM_CHARGE**2)/ (MASS_E * EPSILON_0))
plasma_freq = plasma_ang_freq / (2.0 * np.pi)
print(f"Frequency for plasma at electron temperater at a density of " 
      f"{DENSITY:.1e} m^-3 is: {plasma_freq/1e9:.3f} GHz")

### Electron Collision Frequency


The electron collision frequency is the inverse of the collison time (Freidberg).

$\nu_c = \frac{1}{\tau_c}  = \frac{n\lambda}{T_e^{3/2} 3.44\times 10^6} $. 

This requires $\lambda$, the Coloumb logarithm which is defined as (Freidberg):

$\lambda = 4.9 \times 10^7 \frac{T_e^{3/2}}{n^{1/2}}$ 


In [ ]:
col_log = 4.9e7*temp_e**(3/2)/(np.sqrt(density))

In [ ]:
col_log

# TODO ?????

## Problem 1.2 Thermal Velocities

Calculate thermal velocity (mean speed) for electron and hydrogen ion (proton) at:
* $T = 1keV$
* $T = 10 keV$


Will need the mass of a proton:

* $m_p = 1.6726\times 10^{-27} kg$ [4]

[4] “Proton - Wikipedia.” Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/wiki/Proton

### Will any be realtivistic?

In [ ]:
TEMPERATURES = [1e3, 10e3] # [eV]
MASS_P = 1.6726e-27 # [kg]

In [ ]:
# set up the dataframe
buffer = []
for temp in TEMPERATURES:
    for part in {"electron", "hydrogen"}:
        buffer.append({"particle": part, "temperature [eV]": temp})
df = pd.DataFrame(buffer)
df["mass [kg]"] = df["particle"].apply(lambda p: MASS_P if p == "hydrogen" else MASS_E)

The Maxwell-Boltzmann equation can give us:

* $<v> = \frac{2}{\sqrt{pi}} v_p$
* $v_p = \sqrt{\frac{2k_BT}{m}}$

Where:

* $<v>$ is the average speed
* $v_p$ is the most probable speed
* $k_B$ is the Boltzmann constant [5], which is:


* $k_B = 1.3806\times 10^{-23}\frac{J}{K}$ [6]

[5] “Maxwell–Boltzmann distribution,” Wikipedia. Aug. 19, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Maxwell%E2%80%93Boltzmann_distribution&oldid=1306808676#Typical_speeds

[6] “Boltzmann constant,” Wikipedia. July 11, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Boltzmann_constant&oldid=1300011910#Role_in_the_equipartition_of_energy



In [ ]:
BOLTZMANN = 1.3806e-23 #[j/k]
def get_mean_speed(row):
    temp, mass = row[["temperature [eV]", "mass [kg]"]]
    v_p = np.sqrt((2*BOLTZMANN * temp)/mass)
    return 2/np.sqrt(np.pi)*v_p

df["mean velocity [m/s]"] = df.apply(get_mean_speed, axis=1)

### Is it relativistic?
For this we are saying $v > 0.1c$ is relativistic,
with $c$ being the speed of light:

* $c= 299 792 458 \frac{m}{s}$ [7]

[7] “Speed of light,” Wikipedia. Sept. 15, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Speed_of_light&oldid=1311507394


In [ ]:
SPEED_LIGHT = 299_792_458 #[m/s]

df["frac c"] = df["mean velocity [m/s]"] / SPEED_LIGHT
df["relativistic"] = df["frac c"] > 0.1
df

Therefore, no, none of these particles at these energies are relativistic. The closest is the electron at 10 keV at 0.2% $c$.